In [75]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline

In [77]:
# Import trades database

# Enter your CSV with only the column of results in points. 
# Note that the name of the column must be "points".

df = pd.read_csv('')

df.head()

,Points
0,195
1,160
2,150
3,-55
4,195


In [78]:
# Select positive and negative trades percentage

df.query("Points < 0").count()/df.shape[0], df.query("Points > 0").count()/df.shape[0]


(Points    0.37037
 dtype: float64,
 Points    0.62963
 dtype: float64)

In [80]:
# Sharpe index

df['Points'].mean()/df['Points'].std()


0.3447443947709693

In [81]:
# Profit factor 

pos = df.query("Points > 0")
neg = df.query("Points < 0")

pos['Points'].sum()/-(neg['Points'].sum())


2.192982456140351

In [82]:
# Risk Reward 

rr = pos['Points'].mean()/-neg['Points'].mean()

rr

1.2899896800825594

In [83]:
# Minimum Risk Reward 

w = df.query("Points > 0").count()/df.shape[0]
w = pd.to_numeric(w)

(1 - w)/w

Points    0.588235
dtype: float64

In [84]:
# Mathematical Expectation

In [85]:
# Expected Reward

In [86]:
# Edge

w - (1/(1+rr))


Points    0.192946
dtype: float64

In [87]:
# Kelly 

w - (1-w)/rr

Points    0.342519
dtype: float64